In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jul  8 14:53:57 2022

@author: hannahharris
"""

import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import copy
from gtfparse import read_gtf


import os
os.getcwd()

def get_all_comps(transcript_coords): 
    '''
    takes in a list of anything (transcripts, starts)
    returns a list of all the comparisons  
    '''
    if not transcript_coords:
       return [[]]
    else:
        all_comps = get_all_comps(transcript_coords[:-1])
        all_comps_1 = all_comps + [c + [transcript_coords[-1]] for c in all_comps]
        return all_comps_1 
    
def get_right_length_comps(all_comps):
    '''
    outputs all combos of length=2
    '''    
    return [x for x in all_comps if len(x) == 2]


def get_all_starts(table):
    '''
    take a table (gtf like  format)
    returns dictionnary of genes, starts, and transcripts
    '''
    gene_dict = {}
    for row in range(len(table)):
        try:
            gene_dict[table['gene_name'].iloc[row]][1].add(table['transcript_id'].iloc[row])

            if table['strand'].iloc[row] == '-':
                gene_dict[table['gene_name'].iloc[row]][0].add(table['end'].iloc[row])

            else:
                gene_dict[table['gene_name'].iloc[row]][0].add(table['start'].iloc[row])
        except KeyError:
            if table['strand'].iloc[row] == '-':
                gene_dict[table['gene_name'].iloc[row]] = ({table['end'].iloc[row]}, {table['transcript_id'].iloc[row]})
            else:
                gene_dict[table['gene_name'].iloc[row]] = ({table['start'].iloc[row]}, {table['transcript_id'].iloc[row]})
             
    return gene_dict

def all_transcripts(gene_dict):
    '''
    '''
    new_dict={}
    for key, value in gene_dict.items():
        #this filter subsets it to genes with only one TSS
        if len(value[1])  > 1:
            new_dict[key] = value
    
    return new_dict
        

def make_df_starts(new_dict):
    '''
    takes a dictionary and generates a df of starts with num_start 
    '''
    #so compare to the set dict of ensembl transcripts 
    start_end_to_index_dict = {} # want to have the start/end coord and also the index number 
    
    ens_ids = []
    TSS_number_list  = []
    gene = []
    ix = 0
    for key, values in new_dict.items():
        TSS_number = 1
        for value in values[0]:
            new_df = ercc_new[ercc['gene_name']  == key]
            for row in range(len(new_df)):
                if new_df['start'].iloc[row]  == value:
                    ix +=1
                    ens_ids.append(new_df['transcript_id'].iloc[row])
                    TSS_number_list.append(TSS_number)
                    gene.append(key)
                elif new_df['end'].iloc[row]  == value: 
                    ix +=1
                    ens_ids.append(new_df['transcript_id'].iloc[row])
                    TSS_number_list.append(TSS_number)
                    gene.append(key)
            TSS_number += 1
    
    len(ens_ids)
    len(TSS_number_list)
    zipped  = list(zip(ens_ids, TSS_number_list, gene))
    df_zip = pd.DataFrame(zipped, columns = ['transcript_id', 'TSS', "gene_name"], dtype = str) 
    return df_zip


def subset_ercc(ercc_new, filtering_list, gene_name = True):
    '''
    take in ERCC filters based on whether its in a list 
    '''
    if gene_name:
        test1 = ercc_new[ercc_new.gene_name.isin(filtering_list)]
    else: 
        test1 = ercc_new[ercc_new.transcript_id.isin(filtering_list)]
    
    return test1
    
def get_coords_bed(bed):
        
        bed = bed[bed[0] ==  "chrX"] 
        v2_start = list(bed[1])
        v3_end = list(bed[2])
        zipped_bed = list(zip(v2_start,v3_end))
        
        return zipped_bed
    


In [4]:
if __name__ == "__main__":
    ercc = pd.read_csv("annotation_with_ERCC.txt",sep=' ')
    #changed some gene names in here#
    genes = ["CMV", "EEF1A1", "ACTB", "TXLNG", "TXLNGY", "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y", "PRKX", "PRKY", "NLGN4X", "NLGN4Y"]

    subsetted_xyp = subset_ercc(ercc, genes, gene_name = True)
    subsetted_xyp.to_csv("subsetted_ercc_annotation_1.txt",  index=None, sep=str('\t'))
    subsetted_xyp.iloc[:,0:8].to_csv("subsetted_ercc_annotation_1.txt",  index=None, sep=str('\t'))
        
    subsetted_xyp.iloc[:,[0,3,4]].to_csv("subsetted_ercc_annotation.txt",  index=None, sep=str('\t'))

    all_starts_dict = get_all_starts(subsetted_xyp)

    
    all_starts_dict['USP9Y'] = ({12662368}, {'mine'}) #new USP9Y start 12,662,368

    all_starts_dict['TMSB4Y'] = ({13703899}, {'mine'})  
    
    all_starts_dict['USP9X'] = ({41085445}, {'mine'}) 
    
    all_starts_dict['CMV'] = ({1}, {'mine'}) 
    
    #using the fantom promoters/verified start sites, going to add those in here as 'mine'
    #from /lab/solexa_page/220516_mpra/helen_start_sites/fantom_promtoers
    
    all_starts_dict['DDX3X'] = ({41334161}, {'fantom'}) 
    all_starts_dict['DDX3Y'] = ({12904881}, {'fantom'}) 
    all_starts_dict['EIF1AX'] = ({20141834}, {'fantom'}) 
    all_starts_dict['EIF1AY'] = ({20575775}, {'fantom'}) 
    all_starts_dict['KDM5C'] = ({53225206}, {'fantom'}) 
    all_starts_dict['KDM5D'] = ({19744725}, {'fantom'}) 
    all_starts_dict['KDM6A'] = ({44873187}, {'fantom'}) 
    all_starts_dict['RPS4X'] = ({72277247}, {'fantom'}) 
    all_starts_dict['RPS4Y1'] = ({2841601}, {'fantom'}) 
    all_starts_dict['USP9X'] = ({41085444}, {'fantom'}) 
    all_starts_dict['USP9Y'] = ({12662352}, {'fantom'}) 
    all_starts_dict['UTY'] = ({13479986}, {'fantom'}) 
    all_starts_dict['ZFX'] = ({24149728}, {'fantom'}) 
    all_starts_dict['ZFY'] = ({2935380}, {'fantom'}) 



In [6]:
list_gene = [] 
transcript = [] 
list_starts = [] 
    
for key, val in all_starts_dict.items(): 
    for start in val[0]:
        list_gene.append(key)
        list_starts.append(start)
        to_add = key + "_" + str(start)
        transcript.append(to_add)
            
zipped = list(zip(list_gene, list_starts, transcript))
df_zip = pd.DataFrame(zipped,  columns = ["gene.x","starts_tx", "transcript"], dtype = str) 
df_zip.to_csv("starts.csv")
    

In [7]:
#get distances for each mpra sequence start from the TSS of each gene       
    
    
filename_list = ['lcl_50_1.txt', 'fib_1_50_1.txt', 'fib_2_50_1.txt', 'full_fib_50_1.txt', 'fib_1_200_1.txt', 
                'fib_2_200_1.txt', 'full_fib_200_1.txt']

    
for filename in filename_list:    
    
    df = pd.read_csv(filename, sep = "\t") #     df = pd.read_csv("fib_counts_1_200.txt", sep = "\t") # 

    df[['chr', 'nums']] = df['coords'].str.split(":", expand = True) 
    
    df[['start', 'end']] = df['nums'].str.split("-", expand = True) 
    

    merged = pd.merge(df_zip, df, on='gene.x')
    #print(df)
    merged_minus = merged[merged['orientation.x'] == 'minus']
    
    merged_minus['start'] = merged_minus["start"].astype("float64")  
    merged_minus['starts_tx'] = merged_minus["starts_tx"].astype("float64") 
    merged_minus['distance'] = (merged_minus['start'] - merged_minus['starts_tx'])*(-1) #bc on the - strand

    merged_plus = merged[merged['orientation.x'] == 'plus']
    
    merged_plus['start'] = merged_plus["start"].astype("float64") 
    merged_plus['starts_tx'] = merged_plus["starts_tx"].astype("float64") 
    merged_plus['distance'] = merged_plus['start'] - merged_plus['starts_tx'] 

    #type(merged_minus)
    concat_merged_dis = pd.concat([merged_minus, merged_plus])
    #print(concat_merged_dis)
    concat_merged_dis.to_csv("concat_merge_dist_" + filename + "v1.csv")
    

/tmp/ipykernel_2167452/867867855.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_minus['start'] = merged_minus["start"].astype("float64")
/tmp/ipykernel_2167452/867867855.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_minus['starts_tx'] = merged_minus["starts_tx"].astype("float64")
/tmp/ipykernel_2167452/867867855.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats